In [3]:
import numpy as np
import pandas as pd
import urllib
from html.parser import HTMLParser
from bs4 import BeautifulSoup
from tqdm import tqdm
import pymorphy3

In [4]:
import pymorphy3
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS=='NOUN':
            res.append(p.normal_form)
    return res

# v1 for single region scraping

In [ ]:
region='krasnodar' #'kaluga'# 'sanktpeterburg' ='gornoaltaysk'
period=pd.date_range(start='2022-01-01', end='2022-01-31', freq='1D')
keywords=[]
headers=pd.DataFrame({'date':[1], 'news':[1]})
morph = pymorphy3.MorphAnalyzer()
for date in tqdm(period):
    d,m,y=date.day, date.month, date.year
    url='https://%s.bezformata.com/daynews/?nday=%i&nmonth=%i&nyear=%i'%(region,d,m,y)
    try: 
        f=urllib.request.urlopen(url, data=None, cafile=None, capath=None, cadefault=False, context=None)
    except Exception as e:
        print(date, type(e), e)
        continue
    text=f.read().decode('utf-8')
    soup = BeautifulSoup (text, 'html.parser')
    h=[]
    for t in soup.find_all('h3')[8:]:
        head=t.get_text()
        words=lemmatize(head)
        a=np.isin(keywords, words)
        if sum(a):
            h.append(head)
    if len(h):
        headers=pd.concat([headers, pd.DataFrame({'date':[date.strftime('%Y-%m-%d')]*len(h), 'news':h})], ignore_index=True)
headers.drop([0], inplace=True)
headers
             

100%|██████████| 31/31 [00:13<00:00,  2.34it/s]


,date,news


# v2 for multiple regions from df scraping

In [5]:
regions_df = pd.read_csv('bezformata-oktmo.csv', sep=';', usecols=['Name', 'oktmo'])
regions_df.head()

,Name,oktmo
0,belgorod,14
1,bryansk,15
2,vladimir,17
3,voronej,20
4,moskva,45


In [ ]:
regions = regions_df['Name'] 

morph = pymorphy3.MorphAnalyzer()    

# settings
keywords = []              # заголовки, которые нужно оставить (по вхождению лемм)
exclude_keywords = []      # заголовки, которые нужно исключить (если найдётся хоть одна такая лемма)
trash_words = ['Центр', 'Приволжье', 'Сибирь', 'Урал', 'Юг', 'Северо-Запад', 'Дальний Восток', 'Северный Кавказ']    # not to include in the headlines because of some parsing mistake
period = pd.date_range(start='2022-01-01', end='2022-12-31', freq='1D')

headers = []   
 
# ---------- основной цикл ----------
for region in regions:
    for date in tqdm(period, desc=f'Region: {region}', leave=False):
        d, m, y = date.day, date.month, date.year
        url = f'https://{region}.bezformata.com/daynews/?nday={d}&nmonth={m}&nyear={y}'

        try:
            f = urllib.request.urlopen(url)
        except Exception:
            continue

        text = f.read().decode('utf-8')
        soup = BeautifulSoup(text, 'html.parser')

        for t in soup.find_all('h3'):
            head = t.get_text(strip=True)
            if not head:
                continue

            # приводим заголовок к леммам один раз
            words = lemmatize(head)

            # --- фильтр по исключающим словам ---
            if exclude_keywords and np.isin(exclude_keywords, words).any():
                continue

            # --- фильтр: заголовок не должен состоять только из "пустых" слов ---
            if trash_words and all(w in trash_words for w in words):
                continue

            # --- если keywords пуст ---
            if not keywords:
                headers.append({
                    'region': region,
                    'date': date.strftime('%Y-%m-%d'),
                    'news': head
                })
                continue

            # --- если keywords заданы ---
            if np.isin(keywords, words).any():
                headers.append({
                    'region': region,
                    'date': date.strftime('%Y-%m-%d'),
                    'news': head
                })


# ---------- финальный DataFrame ----------
headers = pd.DataFrame(headers)

In [8]:
headers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3014921 entries, 0 to 3014920
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   region  object
 1   date    object
 2   news    object
dtypes: object(3)
memory usage: 69.0+ MB


In [9]:
headers.to_csv('allNews2022.csv', index=False)

In [10]:
headers

,region,date,news
0,belgorod,2022-01-01,Центр
1,belgorod,2022-01-01,Приволжье
2,belgorod,2022-01-01,Сибирь
3,belgorod,2022-01-01,Урал
4,belgorod,2022-01-01,Юг
...,...,...,...
3014916,grozniy,2022-12-31,Глава ЧР прошелся по центральной площади Грозного
3014917,grozniy,2022-12-31,Р. Кадыров поздравил Информационное управление...
3014918,grozniy,2022-12-31,М. Хучиев награжден орденом «За заслуги перед ...
3014919,grozniy,2022-12-31,В 2022 году в Чеченской Республике отремонтиро...


In [54]:

headers[(headers['region'] == 'moskva') & (headers['date'] == '2022-01-02')].head(20)



,region,date,news
13403,moskva,2022-01-02,Центр
13404,moskva,2022-01-02,Приволжье
13405,moskva,2022-01-02,Сибирь
13406,moskva,2022-01-02,Урал
13407,moskva,2022-01-02,Юг
13408,moskva,2022-01-02,Северо-Запад
13409,moskva,2022-01-02,Дальний Восток
13410,moskva,2022-01-02,Северный Кавказ
13411,moskva,2022-01-02,"«Я рада, что сейчас у нас все так»: Асмус обра..."
13412,moskva,2022-01-02,Украинского футболиста затравили за публикацию...


*approximately 160 mins - for news for all the municipalities within 1 year without keywords filter*

In [5]:
name=region+'_'+headers.date.min()+'_'+headers.date.max()
headers.to_csv(name, index=False)